In [1]:
import pandas as pd

In [2]:
script_df = pd.read_csv('../what-kind-of-bear-is-best/the-office-lines - scripts.csv')

In [3]:
script_df.head()

,id,season,episode,scene,line_text,speaker,deleted
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False


In [30]:
script_df.describe()

,season,episode,scene
count,59909.000000,59909.000000,59909.000000
mean,5.348178,11.558597,27.143852
std,2.389427,6.986208,17.860616
min,1.000000,1.000000,1.000000
25%,3.000000,5.000000,14.000000
50%,5.000000,11.000000,25.000000
75%,7.000000,18.000000,37.000000
max,9.000000,26.000000,116.000000


In [7]:
script_df.set_index('id', inplace=True)

In [29]:
script_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False


Let's see how many characters does the script have

In [11]:
characters = script_df.speaker.drop_duplicates()

In [12]:
len(characters)

797

According to this data source, the office has 797 characters, including minor guests appearances

I've always wanted to see who's got the more lines.

In [33]:
speakers_df = script_df.groupby('speaker')['speaker'].count().sort_values(ascending=False)

In [37]:
speakers_df.head()

speaker
Michael    12137
Dwight      7529
Jim         6814
Pam         5375
Andy        3968
Name: speaker, dtype: int64

seems like Michael has the most lines by a good number, followed by Dwight, Jim and Pam. That sounds about right.

### Now I want to find a set of questions, where Dwight provides an answer to them.

In [49]:
questions_df = script_df.loc[script_df['line_text'].str.contains('.*\?$')]

In [50]:
questions_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
7,1,1,3,"I've, uh, I've been at Dunder Mifflin for 12 y...",Michael,False
10,1,1,3,What?,Pam,False
11,1,1,3,Any messages?,Michael,False


In [56]:
questions_df.index[:-1]

Int64Index([    1,     3,     7,    10,    11,    25,    32,    34,    37,
               39,
            ...
            59836, 59838, 59864, 59865, 59866, 59869, 59894, 59897, 59903,
            59904],
           dtype='int64', name='id', length=11379)

In [64]:
answers_df = script_df.loc[questions_df.index[:-1]+1] 

In [65]:
answers_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
8,1,1,3,Well. I don't know.,Pam,False
11,1,1,3,Any messages?,Michael,False
12,1,1,3,"Uh, yeah. Just a fax.",Pam,False


In [66]:
dwight_answers_df = answers_df.loc[answers_df['speaker'] == 'Dwight']

In [67]:
dwight_answers_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
86,1,1,19,What?,Dwight,False
88,1,1,19,Just clearing my desk. I can't concentrate.,Dwight,False
93,1,1,19,Safety violation. I could fall and pierce an o...,Dwight,False
96,1,1,20,Downsizing? I have no problem with that. I hav...,Dwight,False
125,1,1,26,On his mother's grave.,Dwight,False


In [68]:
len(dwight_answers_df)

1363

In [69]:
questions_df = questions_df.loc[dwight_answers_df.index-1]

In [70]:
questions_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
85,1,1,19,"Sure. Mr. Davis, let me call you right back. Y...",Jim,False
87,1,1,19,What are you doing?,Jim,False
92,1,1,19,Why not?,Jim,False
95,1,1,19,Downsizing?,Dwight,False
124,1,1,26,Can you promise that?,Stanley,False


In [80]:
questions_and_answers_df = pd.concat([questions_df,dwight_answers_df])

In [81]:
questions_and_answers_df.sort_index()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
85,1,1,19,"Sure. Mr. Davis, let me call you right back. Y...",Jim,False
86,1,1,19,What?,Dwight,False
87,1,1,19,What are you doing?,Jim,False
88,1,1,19,Just clearing my desk. I can't concentrate.,Dwight,False
92,1,1,19,Why not?,Jim,False
...,...,...,...,...,...,...
59652,9,23,47,"Mose! What..ooh, Mose. [Mose opens trunk]",Dwight,False
59737,9,23,68,You ready?,Jim,False
59738,9,23,68,[chuckles] You kidding? I was born ready. [mim...,Dwight,False
